In [0]:
%python

dbutils.widgets.text("path", "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/users-historical")
dbutils.widgets.text("catalog", "test_catalog")
dbutils.widgets.text("schema", "bronze")

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS test_catalog.bronze;

In [0]:
%sql
-- List all available catalogs
SHOW CATALOGS;

-- Use the desired catalog if it exists
USE CATALOG test_catalog;
USE SCHEMA bronze;

In [0]:
%sql
SELECT current_catalog(), current_database()


In [0]:
%sql

LIST '${path}'

In [0]:
%sql
SELECT *
FROM parquet .`dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/users-historical`
LIMIT 10;

In [0]:
%sql
SELECT *
FROM read_files(
  'dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/users-historical',
  format => 'parquet'
)
LIMIT 10;

In [0]:
DROP TABLE IF EXISTS test_catalog.default.historical_users_rf;

USE CATALOG test_catalog;
USE SCHEMA bronze;

CREATE TABLE IF NOT EXISTS historical_users_rf
SELECT *
FROM read_files(
  'dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/users-historical',
  format => 'parquet'
);

-- preview
SELECT *
FROM historical_users_rf
LIMIT 10;

In [0]:
DESCRIBE TABLE EXTENDED historical_users_rf;

In [0]:
%python
# read source
df = (
    spark
    .read
    .format('parquet')
    .load('dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/users-historical')
)
# write table
(
    df
    .write
    .mode('overwrite')
    .saveAsTable('test_catalog.bronze.historical_users_py')
)
# review table
historical_users = spark.table('test_catalog.bronze.historical_users_py')
historical_users.show(5)

In [0]:
-- clean up
DROP TABLE IF EXISTS historical_users_ci;
-- create table
CREATE TABLE historical_users_ci (
  user_id STRING,
  user_first_touch_timestamp BIGINT
);
-- COPY INTO table using path from widget
COPY INTO historical_users_ci
FROM '${path}'
FILEFORMAT = PARQUET
COPY_OPTIONS (
  'mergeSchema' = 'true'
);

In [0]:
-- COPY INTO table again with unchanged src data
COPY INTO historical_users_ci
FROM '${path}'
FILEFORMAT = PARQUET
COPY_OPTIONS (
  'mergeSchema' = 'true'
);